In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write
from ase.calculators import lj, eam
from ase.optimize import BFGS, LBFGS

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
data_dump = WidgetDataDumper(prefix="module_05")
display(data_dump)

In [ ]:
module_summary = Textarea("general comments on this module", layout=Layout(width="100%"))
data_dump.register_field("module-summary", module_summary, "value")
display(module_summary)

_Reference textbook / figure credits: Charles Kittel, "Introduction to solid-state physics", Chapters 20-21; Wikipedia_

# Point defects

Point defects are the simplest kind of imperfections that can be found in the structure of a crystalline material. _Vacancies_ are empty lattice sites, _interstitials_ are additional atoms that fill a void in the structure.  Impurities can also occur in the form of _substitutional_ atoms, that replace some of the atoms of the main lattice. Defects can have a large impact on the electronic properties of a material, drive transport of ions, and are often related to solid-state transformations. 

Particulary for ionic crystals, where defects must preserve  overall charge neutrality, defects must come in pairs: _Frenkel defects_ are a vacancy-interstitial pair (conceptually corresponding to an atom moved from a lattice to an interstitial site), _Schottky defects_ are paris of vacancies formed by atoms with opposite charge. 

<img src="figures/defecttypes.png" width="600"/>

In the vast majority of cases the presence of a defect results in the *increase* of the energy of the crystal by an amount $\Delta E$. Then why are there defects in solids? The answer involves a combination of thermodynamics and kinetics. You may recall that the stability of a system at constant temperature $T$ and pressure is determined by its *Gibbs free energy* $G=H-TS$, where $H$ is the enthalpy (which to a very good approximation is equal to the internal energy for a solid at ambient pressure) minus a term that involves the *entropy* of the system. 

A full discussion of the microscopic definition of entropy is beyond the scope of this module, but we will just say that one can define it as $S=k_B \ln \Omega$, where $k_B=1.38\cdot 10^{-23}$ J/K, and $\Omega$ is the number of microscopic configurations that are consistent with the macroscopic observables and boundary conditions. For a lattice with $N$ sites and $M$ vacancies, one can estimate $\Omega$ by combinatorial arguments

$$
\ln \Omega = \frac{N!}{M!(N-M)!} \approx N \ln N - M\ln M -(N-M)\ln (N-M)
$$

where we also used Stirling approximation $\ln N! \approx N \ln N$. 

By ignoring constant terms that depend only on $N$, and rewriting in terms of the vacancy concentration $x=m/N$, one can find $N^{-1}\ln\Omega \approx -x\ln x -(1-x)\ln(1-x)$. Thus, the molar free energy $G/N$ of a solid with a concentration of vacancies $x$ can be written as 

$$
g(x) =  x \Delta E  + k_B T\left[ x\ln x + (1-x)\ln(1-x)\right]
$$

<span style="color:blue">**01** The equilibrium concentration of vacancies can be obtained differentiating the expression and solving for $x$. Write the analytical expression and verify it using the interactive widget. What is the highest molar vacancy concentration that is theoretically possible? </span>

In [ ]:
ex1_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex1-answer", ex1_txt, "value")
display(ex1_txt)

In [ ]:
# TODO AG: add a widget here with two panels: left panel shows the molar free 
# energy as a function of x for a given Delta E and T. right panel shows the curve 
# for the equilibrium vacancy concentration (x vs T). Add a dot at the minimum of g(x)
# and at the point that correspond to the equilibrium concentration for the selected temperature
# parameters are Delta and T
# use ranges that are reasonable for a metal, say delta between 0.1 and 2 eV, and T ranging from 0 to 4000K

<span style="color:blue">**02** Do you think that it would be possible, in practice, to reach the maximum vacancy concentration? What might happen before that? </span>

In [ ]:
ex2_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex2-answer", ex2_txt, "value")
display(ex2_txt)

<span style="color:blue">**03** Metals at room temperature often have a much higher vacancy concentration than it would be predicted by the expressions above. How can you explain this observation? </span>

In [ ]:
ex3_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex3-answer", ex3_txt, "value")
display(ex3_txt)

# Vacancy formation energy

In order to compute the vacancy formation energy, we need first to compute the energy of an atom in a perfect crystal. If the calculation is performed for a supercell with $n$ atoms, the molar energy is the total potential for the cell divided by $n$, $\epsilon = E_n / n$. 

One then needs to build a supercell (that could contain a different number $m$ of atoms) and remove one. The energy of a vacancy can be computed by subtracting from the energy of the supercell that of $m-1$ atoms of a perfect crystal, 

$$ 
\Delta E = E_{m-1} - (m-1)\epsilon
$$

The visualizer below displays green points at the _fcc_ lattice parameters, and green spheres where an Al atom sits. See how one of the Al atoms is missing. The second frame in the viewer contains only Al atoms, so the vacancy is harder to see. 

In [ ]:
# AG: have a chemiscope viewer that displays a 2x2x2 cell with an atom missing. Use both mg atoms to show the full lattice, 
# and Al atoms to show where atoms actually are. a second frame should have only aluminum

<span style="color:blue">**04** Write a function that computes the vacancy formation energy of aluminum, using a conventional _fcc_ cell (4-atoms, lattice parameter $a_0 = 4.05$Å) to get the molar energy, and a supercell replicated $M$ times in each direction, from which you can then remove one atom. Return the vacancy formation energy computed from the formula above.  </span>

_NB: you can remove an atom from an `ase.Atoms` structure by calling `del structure[[index]]` (note the double brackets). See the [module on interatomic potentials](./04-Potentials.ipynb) to remind you how the ASE potential calculator works_

In [ ]:
# AG: make a WidgetCodeInput with already the commands to initialize the EAM calculator. 
# it should take the number of replications nrep, compute vacancy formation energy and return it.
# The widged should have a demonstrator with a slider that goes from 2 to 5 nrep, and when you
# move the slider it will compute and print delta E

<span style="color:blue">**05** Compute the vacancy energy for different cell size. Why does it change? How large should the supercell be to converge the energy within 5 meV?   </span>

In [ ]:
ex5_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex5-answer", ex5_txt, "value")
display(ex5_txt)

# Energy relaxation

The exercise above assumes that atoms in the vicinity of a vacancy remain in their ideal lattice position. This is obviously a harsh approximation: with the change in environment, atoms will rearrange to find a more favorable position. This will both change the structure, and the energy of the system, and therefore affect the vacancy formation energy. 

To account for this rearrangement, one can _relax_ the geometry of the vacancy structure. The idea is simply to minimize the potential energy $V$ of the system, looking for a structure for which the force $\mathbf{f}=-\nabla V$ is zero. There is a large number of schemes that have been proposed to achieve this goal (see here those that are [implemented in ASE](https://wiki.fysik.dtu.dk/ase/ase/optimize.html)) which underscores the importance of energy relaxation to compute accurately the stability of structures. 

The most naïve approach, called [steepest descent](https://en.wikipedia.org/wiki/Gradient_descent) iteratively optimizes the structure following the gradient, 

$$
\mathbf{r} \leftarrow \mathbf{r} -\alpha \nabla V(\mathbf{r}).
$$

Even though it may seem to be an efficient idea (following the direction of maximum decrease of the potential) in a high-dimensional problem it leads to very slow convergence. If you are curious, you can read about the mathematical aspects of optimization, and why steepest descent does not work, in these excellent [lecture notes](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf). 

Here we will use a rather sophisticated method, named [BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm) from the names of its inventors. 
In practice, a BFGS optimization can be run as follows:

```python
structure = Atoms( ... , calculator=...)   # you can also initialize the structure and set the calculator with structure.calc = ...
opt = BFGS(structure)
opt.run(fmax=0.01)
```


We will start by looking at a pre-coded function that minimizes the energy of a Lennard-Jones dimer. 

In [ ]:
#AG: write a stub of a function, that save the optimization trajectory to a file. Use ase.opt.BFGS 
# The demo should load and visualize that file in chemiscope, with the energy vs iteration as a property panel

<span style="color:blue">**06** Read the function above, that takes as arguments the parameters of a LJ potential, initializes a dimer at separation equal to $\sigma$, and runs a geometry optimization.  Observe in the visualizer the behavior of the energy and distance. How many steps are needed to convergence (set $\sigma=1, \epsilon=1$ to ensure reproducibility) ? How many steps are needed if you make the convergence threshold 10 times larger and smaller? </span>

In [ ]:
ex6_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex6-answer", ex6_txt, "value")
display(ex6_txt)

# Decohesion curves

The surface energy of a material $\gamma$ is the energy that one has to pay to create a unit area of surface. It is an important parameter that has implications for fracture mechanics, tribology, and adhesion. 

<img src="figures/surface-energy-2.png" width="600"/>

In a computational model, one can realize quite easily the thought experiment of achieving a clear-cut cleavage of a crystal along a high-symmetry surface, using exactly the same strategy discussed in [the crystallography module](./02-Crystallography.ipynb) to generate a surface: a bulk structure is formed with a unit cell aligned along appropriate directions, and then one of the lattice parameter is extended, to create a slab geometry, with _two_ surfaces separated by vacuum. 

By computing the energy as a function of separation, we can build _decohesion curves_ that give a cue on the magnitude of the surface energy. The curve is usually expressed such that the energy for the ideal crystal structure is zero, and the energy at large separation corresponds to the surface energy. 

<span style="color:blue">**07a** Modify this function so that it returns a decohesion curve, expressed as a surface energy in eV/Å², and baselined in such a way that the large-separation value corresponds to the surface energy $\gamma$.  </span>

In [ ]:
# AG: the function should already build a large-ish (say 4x4x4) supercell of Al, along the (111,01-1, 211) axes.
# It should get just one option, a drop-down box picking the direction. It should return the energy curve and the frames, 
# and these should be visualized by the demo as a chemiscope. They will only need to subtract the zero and compute the area
# to return things in eV/A^2. add plenty of comments

<span style="color:blue">**07b** What is the surface energy of aluminum along the (111), (01-1), (211) directions? </span>

In [ ]:
ex7_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex7-answer", ex7_txt, "value")
display(ex7_txt)

# Dislocations

**AG: REMOVE THIS SECTION AFTER HAVING USED WHAT IS USEFUL - THIS IS TOO COMPLICATED AND DOESN'T REALLY WORK WELL**

Dislocations are line defects that can be understood as a loss of register between lattice planes - essentially one section of a plane of atoms is removed, and the remaining pieces are glued together. 
Dislocations can move when a material is subject to strain, and are involved in the mechanism that underlie plastic deformations in metals. The mechanisms by which dislocations are created in a real material can be [quite complicated](https://en.wikipedia.org/wiki/Frank%E2%80%93Read_source).  

<img src="figures/dislocation-edge.png" width="400"/>

<div style='text-align:center; font-style:italic'>
   Credits: Wikipedia, user Magasjukur2, License: CC-BY-SA
</div>


Even considering the idealized scenario in which a lattice plane is removed to form an edge dislocation, the image above is highly unrealistic. Atoms around a dislocation relax to form a distorted structure (_dislocation core_) surrounded by a long-range elastic field, whose form can be evaluated analytically based on continuum elasticity theory. The image below shows a relaxed edge dislocation in an _fcc_ material: if you don't trace the [Burgers circuit](https://en.wikipedia.org/wiki/Burgers_vector), it is actually quite hard to notice that the structure below is not a perfect lattice

<img src="figures/dislocation-img.png" width="400"/>

<span style="color:blue">**08a** The function below generates a dislocation pair model, by removing one half of a (111) lattice plane and then relaxing the geometry. The relaxation curve is visualized in the demo widget. Study the function to understand what is going on. Modify the function so that it returns the line energy of the dislocation (energies are in eV, lengths in Å). </span>

In [ ]:
#ag: I already made a stub of what I'd like to have here:
# basically this code (which you can also use for some of the ex above)
# needs to be encapsulated in a function, so they can see it. parameters can be the
# repeat counts (set the minimum at 4x4x1) and make sure that the optimization log
# is visible in the output box. return the raw energy, the only thing they'll have to do is 
# modify to get the line energy instead.

In [ ]:
#calc = eam.EAM(potential='data/Al99.eam.alloy')
calc = lj.LennardJones(sigma=2.62, epsilon=0.41, rc=2*2.62)

In [ ]:
a0 = 4.04
h0 = np.sqrt(np.asarray([[3,0,0],[0,3/2,0],[0, 0, 1/2]]))
pos0 = np.sqrt(np.asarray([[ 0, 0, 0],[0,9/24, 1/8.],[1/3,16/24,0],[1/3,1/24,1/8],[4/3,4/24,0],[4/3,25/24,1/8]]))
struc = ase.Atoms("Al6", cell=a0*h0, positions=pos0*a0, pbc=True  )
struc.calc = calc
e0 = struc.get_potential_energy()/6

In [ ]:
a0 = 4.04
h0 = np.sqrt(np.asarray([[3,0,0],[0,3/2,0],[0, 0, 1/2]]))
pos0 = np.sqrt(np.asarray([[ 0, 0, 0],[0,9/24, 1/8.],[1/3,16/24,0],[1/3,1/24,1/8],[4/3,4/24,0],[4/3,25/24,1/8]]))
struc = ase.Atoms("Al6", cell=a0*h0, positions=pos0*a0, pbc=True  )
struc.calc = calc
e0 = struc.get_potential_energy()/6

In [ ]:
struc2d=struc.repeat((16,32,1))
# atoms to be removed
sel = np.where((np.abs(struc2d.positions[:,0]-struc2d.cell[0,0]/2)< a0*np.sqrt(3)/2+ 1e-5) & (np.abs(struc2d.positions[:,1]-struc2d.cell[1,1]/2)<struc2d.cell[1,1]/4+1e-3) )[0]
del(struc2d[sel])
# pulls atoms closer
sel = np.where((np.abs(struc2d.positions[:,1]-struc2d.cell[1,1]/2)<struc2d.cell[1,1]/4+1e-3) )[0]
xx = struc2d.positions[sel]
struc2d.positions[sel,0] -= (
    0.9*a0*np.sign(xx[:,0]-struc2d.cell[0,0]/2)*
    np.exp(-(xx[:,0]-struc2d.cell[0,0]/2)**2*0.5/(4*a0)**2)*
    np.exp(-(xx[:,1] -struc2d.cell[1,1]/2)**2*0.5/((struc2d.cell[1,1]/3)**2))
)
     #          np.abs(struc2d.positions[:,0]-struc2d.cell[0,0]/2)< a0*np.sqrt(3)+ 1e-5)

struc2d.calc = calc

In [ ]:
opt = LBFGS(struc2d, trajectory='dislocation.xyz')
opt.run(fmax=0.002)

In [ ]:
opt_traj = read('dislocation.xyz',':')[0:400]

In [ ]:
chemiscope.show(properties=dict(index=np.arange(len(opt_traj)),energy=[(f.calc.results['energy']-e0*len(f))/(2*f.cell[2,2]) for f in opt_traj]),
                frames=opt_traj, settings= {'structure' : [{'unitCell': True, 'spaceFilling': False, 'supercell':{'0':1,'1':1,'2':1}}]})

In [ ]:
opt_traj[0].cell

<span style="color:blue">**08b** Run the simulations with different supercell sizes (use nz </span>

In [ ]:
ex8_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex8-answer", ex8_txt, "value")
display(ex8_txt)

In [ ]:
a0 = 4.04
h0 = np.sqrt(np.asarray([[3,0,0],[0, 1/2, 0],[0,0,3/2]]))
pos0 = np.sqrt(np.asarray([[ 0, 0, 0],[0,1/8, 9/24],[1/3,0,16/24],[1/3,1/8,1/24],[4/3,0,4/24],[4/3,1/8,25/24]]))
struc = ase.Atoms("Al6", cell=a0*h0, positions=pos0*a0, pbc=True  )
struc.calc = calc
e0 = struc.get_potential_energy()/6

In [ ]:
struc2d=struc.repeat((16,32,1))
ref = struc2d.copy()
# atoms to be removed
sel = np.where((np.abs(struc2d.positions[:,1]-struc2d.cell[1,1]/2-a0*0.12)< a0*0.35+ 1e-5) & (np.abs(struc2d.positions[:,0]-struc2d.cell[0,0]/2)<struc2d.cell[0,0]/4+1e-3) )[0]
del(struc2d[sel])
# pulls atoms closer
sel = np.arange(len(struc2d))
xx = struc2d.positions[sel]
struc2d.positions[sel,1] -= (
    0.3*a0*np.sign(xx[:,1]-struc2d.cell[1,1]/2)*
    np.exp(-(xx[:,1]-struc2d.cell[1,1]/2)**2*0.5/(2*a0)**2)*
    np.exp(-(xx[:,0] -struc2d.cell[0,0]/2)**2*0.5/((struc2d.cell[0,0]/6)**2))
)
     #          np.abs(struc2d.positions[:,0]-struc2d.cell[0,0]/2)< a0*np.sqrt(3)+ 1e-5)

struc2d.calc = calc

In [ ]:
chemiscope.show(frames=[struc2d], mode="structure",
                settings= {'structure' : [{'unitCell': True, 'spaceFilling': False, 'supercell':{'0':1,'1':1,'2':1}}]})

In [ ]:
opt = LBFGS(struc2d, trajectory='dislocation.xyz')
opt.run(fmax=0.001)

In [ ]:
opt_traj = read('dislocation.xyz',':')[0:400]

In [ ]:
chemiscope.show(properties=dict(index=np.arange(len(opt_traj)),energy=[(f.calc.results['energy']-e0*len(f))/(2*f.cell[2,2]) for f in opt_traj]),
                frames=opt_traj, settings= {'structure' : [{'unitCell': True, 'spaceFilling': False, 'supercell':{'0':1,'1':1,'2':1}, 
   'keepOrientation': True,
}]})

In [ ]:
cs.settings

In [ ]:
ref.symbols="Mg"*len(ref)
ref.positions[:,2]+=ref.cell[2,2]

In [ ]:
chemiscope.show(frames=[opt_traj[200]+ref], mode="structure",
                settings= {'structure' : [{'unitCell': True, 'spaceFilling': False, 'supercell':{'0':1,'1':1,'2':1}}]})